In [21]:
def state_to_string(lst):
    string=''
    for i in lst:
        string+=i
    return string

In [22]:
def player_move(state):
    num = state.count('-')
    if num % 2 != 0:
        return 'O'
    else:
        return 'X'

In [23]:
def is_terminal(state):
    winning_conditions = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Horizontal Win
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Vertical Win
        [0, 4, 8], [2, 4, 6]              # Diagonal Win
    ]

    for condition in winning_conditions:
        if state[condition[0]] == state[condition[1]] == state[condition[2]] != '-':
            return True, 1 if state[condition[0]] == 'X' else -1

    if '-' not in state:
        return True, 0

    return False, 0


In [25]:
tree = {}
terminal = []
child_parent = {}
Scores = {}
def create_tree(state = ['-']*9):
    state_in_string = state_to_string(state)
    tree[state_in_string] = []
    move = player_move(state)

    if not is_terminal(state)[0]:

        for i in range(9):
            copy_of_state = state.copy()
            if state[i] == '-':
                copy_of_state[i] = move
                state_copy_string = state_to_string(copy_of_state)
                tree[state_in_string].append(state_copy_string)

                if state_copy_string not in child_parent.keys():
                    child_parent[state_copy_string] = []
                score = is_terminal(state_copy_string)[1]
                Scores[state_copy_string]=[score, move]
                if state_in_string not in child_parent[state_copy_string]:
                    child_parent[state_copy_string].append(state_in_string)


                create_tree(copy_of_state)
    else:
        if state_in_string not in terminal:
            terminal.append(state_in_string)


create_tree()
child_parent['---------']=[None]
Scores['---------']=[0, 'O']

In [26]:
def MiniMax(parents):
    while parents[0] is not None:
        for j in parents:
            score_list=[Scores[k][0] for k in tree[j]]  # using list comprehension it help us to make code compact
            if Scores[j][1] == 'X':
                Scores[j][0] = min(score_list)
            else:
                Scores[j][0] = max(score_list)
            parents = child_parent[j]
            MiniMax(parents)

def update_values():
    for i in terminal:
        parents = child_parent[i]
        MiniMax(parents)

update_values()

In [37]:
import json
with open('Tree-minmax.json', 'w+') as f:
    json.dump(tree, f)

In [27]:
def position_to_move(state, action):
    for i in range(9):
        if state[i] == '-' and action[i] != '-':
            return i

def best_move(state):
    state_str = state_to_string(state)
    children = tree[state_str]
    score_dict = {Scores[i][0]: i for i in children}  # Using dictionary comprehension it help us to make code compact

    best_score = min(score_dict)
    best_move_index = score_dict[best_score]
    return position_to_move(state_str, best_move_index)

In [28]:
def human_move_index():
    row = int(input("Enter the row [1,2,3]:- "))
    column = int(input("Enter the column [1,2,3]:- "))
    if row == 1:
        index = column - 1
    elif row == 2:
        index = 2 + column
    else:
        index = 5 + column
    return index


In [29]:
def display(state):
    print('-------------------')
    print('| ',state[0],' | ',state[1],' | ',state[2],' |')
    print('-------------------')
    print('| ',state[3],' | ',state[4],' | ',state[5],' |')
    print('-------------------')
    print('| ',state[6],' | ',state[7],' | ',state[8],' |')
    print('-------------------')

In [30]:
def play_Game():
    state = ['-']*9
    display(state)
    player = 0
    while not is_terminal(state)[0]:
        if player % 2 != 0:
            index = human_move_index()
            state[index] = 'X'
        else:
            index = best_move(state)
            state[index]='O'
        display(state)
        player += 1

        terminal = is_terminal(state)
        if terminal[1] == -1:
            return 'AI won'
        elif terminal[1] == 1:
            return 'You won'
        elif terminal[0] and terminal[1] == 0:
            return 'Match draw'


# play_Game()

In [ ]:
play_Game()